In [1]:
import numpy as np
import glob
from osgeo import gdal, osr
import pyproj
from affine import Affine
from shutil import copyfile

In [2]:
tfw_file = r"D:\FloodChange\GeneralDownload\48157C0305L.tfw"
base_file = r"D:\FloodChange\BaseRaster\BaseTest.tif"

In [7]:
input_epsg = "EPSG:3857"
output_epsg = "EPSG:32615"
output_esri = "ESRI:102740"

In [8]:
def get_geotransform_from_tfw(tfw_file):
    # READ TFW FILE AND RETURN GDAL GEOTRANSFORM
    with open(tfw_file, 'r') as f:
        tfw = [float(line) for line in f if line is not "\n"]

    geotransform = (tfw[4], tfw[0], tfw[1], tfw[5], tfw[2], tfw[3])
    
    # CHECK LAST OBSERVATION, IF IT'S BIGGER THAN 10e7 IT'S PROLLY STATE PLANE, ELSE UTM
    if np.abs(tfw[5]) > 1e7 :
        outcrs = "ESRI:102740"
    else:
        outcrs = "EPSG:32615"
    return geotransform, outcrs

def reproject_geotransform(geotransform, src_crs, dst_crs):
    src_proj = pyproj.Proj(src_crs)
    dst_proj = pyproj.Proj(dst_crs)
    
    #
    # Create a transformation object to convert between the input and output CRS
    transformer = pyproj.Transformer.from_crs(src_crs, dst_crs)

    # Transform the coordinates of the upper-left and lower-right corners of the input image
    upper_left = transformer.transform(geotransform[0], geotransform[3])
    lower_right = transformer.transform(geotransform[0] + geotransform[1] * geotransform[2],
                              geotransform[3] + geotransform[5] * geotransform[4])
    
    reprojected_geotransform = [upper_left[0], geotransform[1], geotransform[2], upper_left[1], geotransform[4], geotransform[5]]
    
    return reprojected_geotransform

def get_affine_from_geotransform(geotransform):
    return Affine.from_gdal(*geotransform)

def AffineFromTFW(tfw_file, new_crs, old_crs=None):
    geotransform, detected_crs = get_geotransform_from_tfw(tfw_file)
    if old_crs is None:
        old_crs = detected_crs
    new_geotransform = reproject_geotransform(geotransform, old_crs, new_crs)
    affine = get_affine_from_geotransform(new_geotransform)
    return affine
    
def AffineFromTIF(tif_file, new_crs):
    raster_ds = gdal.Open(tif_file)
    geotransform = raster_ds.GetGeoTransform()
    old_crs = raster_ds.GetProjection()
    new_geotransform = reproject_geotransform(geotransform, old_crs, new_crs)
    affine = get_affine_from_geotransform(new_geotransform)
    return affine

def getMatrixFromAffine(affine_transform):
    mt = affine_transform.to_gdal()
    
    matrix = np.array([
        [mt[1], mt[2], mt[0]],
        [mt[4], mt[5], mt[3]],
        [0, 0, 1]
    ])
    
    
    # matrix = np.array(coefficients_tuple).reshape((2, 3))
    # matrix = np.vstack([matrix, [0, 0, 1]])
    return(matrix)

def combineAffine(src_affine, dst_affine):
    return ~src_affine * dst_affine
    

In [14]:
dataloc = r"D:\FloodChange\GeneralDownload\\"
outloc = r"D:\FloodChange\TrainDataset\\"

base_affine=AffineFromTIF(base_file, output_epsg)
files = glob.glob(f"{dataloc}*.tfw")
print(len(files))

for tfw_file in files:
    print(tfw_file)
    # GET FILENAME
    filename = tfw_file.split("\\")[-1]
    filename = filename[:-4]
    
    # CALCULATE AFFINE PARAMETERS
    tfw_affine = AffineFromTFW(tfw_file, output_epsg, old_crs=None)
    combined = combineAffine(base_affine, tfw_affine)
    print(combined)
    combined_matrix = getMatrixFromAffine(combined).flatten()[:6]
    tfw_affine_matrix = getMatrixFromAffine(tfw_affine).flatten()[:6]
    
    # SAVE AFFINE TRANSFORMATION
    #text_file = open(f"{outloc}{filename}.affine", "w")
    #matrix = [f"{l2}," for l2 in combined_matrix]
    #print(matrix)
    # n = text_file.write(matrix)
    
    np.save(f"{outloc}{filename}_affine", combined_matrix)
    np.save(f"{outloc}{filename}_affine_abs", tfw_affine_matrix)
    
    # SAVE 
    copyfile(f"{dataloc}{filename}.tif", f"{outloc}{filename}.tif")
    

110
D:\FloodChange\GeneralDownload\48157C0285L.tfw
| 0.50, 0.00, 6939.14|
|-0.00, 0.50, 12143.16|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48157C0305L.tfw
| 0.50, 0.00, 8150.37|
|-0.00, 0.50, 12169.14|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0170L.tfw
| 0.10,-0.00,-93.55|
| 0.00, 0.10, 2319.54|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0195L.tfw
| 0.10,-0.00, 2288.06|
| 0.00, 0.10, 2379.66|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0195M.tfw
| 0.15,-0.00, 2286.96|
| 0.00, 0.15, 2386.78|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0235L.tfw
| 0.10,-0.00, 7142.11|
|-0.00, 0.10, 1105.81|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0240L.tfw
| 0.10,-0.00, 5905.62|
| 0.00, 0.10, 2464.58|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0245L.tfw
| 0.15,-0.00, 7111.52|
| 0.00, 0.15, 2491.13|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0265L.tfw
| 0.10,-0.00, 8317.56|
| 0.00, 0.10, 2517.99|

| 0.10,-0.00, 9349.18|
| 0.00, 0.10, 10857.72|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0880L.tfw
| 0.10, 0.00, 10587.50|
| 0.00, 0.10, 9496.99|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0885L.tfw
| 0.10,-0.00, 11797.07|
|-0.00, 0.10, 9521.15|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0885M.tfw
| 0.15, 0.00, 11796.62|
| 0.00, 0.15, 9528.86|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0890L.tfw
| 0.10,-0.00, 10559.40|
|-0.00, 0.10, 10882.52|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0895L.tfw
| 0.10, 0.00, 11800.05|
| 0.00, 0.10, 10906.74|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0895M.tfw
| 0.15, 0.00, 11769.28|
| 0.00, 0.15, 10914.44|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0905L.tfw
| 0.15, 0.00, 13007.60|
| 0.00, 0.15, 9545.83|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0905M.tfw
| 0.15, 0.00, 13006.09|
| 0.00, 0.15, 9552.42|
| 0.00, 0.00, 1.00|
D:\FloodChange\Gen

In [12]:
print(np.load(r"D:\FloodChange\TrainDataset\48157C0285L.affine.npy"))

[ 5.00036879e-01  3.86553684e-05  6.93913722e+03 -1.43308306e-05
  5.00020619e-01  1.21431562e+04]
